In [0]:
#Cancellazione delle cartelle precedentemente create
import os, shutil
folder = 'semantic_drone_dataset/'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [1]:
!mkdir -p /content/IPCV
!git clone https://github.com/qubvel/segmentation_models.git /content/IPCV
!mkdir -p /content/lib
!git clone https://github.com/ilmissigno/IPCV.git /content/lib
#Caricamento delle api di kaggle -- N.B Scaricare da sopra a kaggle
#in my account , create my api key per andare a scaricare il file kaggle.json
from google.colab import files
!pip install -q kaggle
uploaded = files.upload() #Caricarlo qui selezionandolo da file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
#Caricamento del dataset da kaggle
!kaggle datasets download -d bulentsiyah/semantic-drone-dataset
#Unzip del dataset e rimozione del zip
!unzip semantic-drone-dataset.zip -d .
!rm -rf semantic-drone-dataset.zip

Cloning into '/content/IPCV'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 922 (delta 13), reused 9 (delta 2), pack-reused 886
Receiving objects: 100% (922/922), 1.82 MiB | 2.05 MiB/s, done.
Resolving deltas: 100% (512/512), done.
Cloning into '/content/lib'...
remote: Enumerating objects: 230, done.
remote: Counting objects: 100% (230/230), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 230 (delta 101), reused 177 (delta 53), pack-reused 0
Receiving objects: 100% (230/230), 3.93 MiB | 3.14 MiB/s, done.
Resolving deltas: 100% (101/101), done.


Saving kaggle.json to kaggle.json
kaggle.json
100% 3.83G/3.83G [01:14<00:00, 58.3MB/s]
100% 3.83G/3.83G [01:14<00:00, 55.4MB/s]
Archive:  semantic-drone-dataset.zip
  inflating: ./semantic_drone_dataset/label_images_semantic/000.png  
  inflating: ./semantic_drone_dataset/label_images_semantic/001.png  
  inflating: ./semantic_drone_dataset/label_images_semantic/002.png  
  inflating: ./semantic_drone_dataset/label_images_semantic/003.png  
  inflating: ./semantic_drone_dataset/label_images_semantic/004.png  
  inflating: ./semantic_drone_dataset/label_images_semantic/005.png  
  inflating: ./semantic_drone_dataset/label_images_semantic/006.png  
  inflating: ./semantic_drone_dataset/label_images_semantic/008.png  
  inflating: ./semantic_drone_dataset/label_images_semantic/011.png  
  inflating: ./semantic_drone_dataset/label_images_semantic/013.png  
  inflating: ./semantic_drone_dataset/label_images_semantic/014.png  
  inflating: ./semantic_drone_dataset/label_images_semantic/015.p

In [2]:
import lib.dataset_creator as d
d.train_test_val(60,20,20,100)
d.data_augment(1000,4000,6000)
d.data_augment_val(1000,4000,6000)
d.data_augment_test(1000,4000,6000)

100
100


Using TensorFlow backend.
100%|██████████| 24/24.0 [00:00<00:00, 44.03it/s]
100%|██████████| 24/24.0 [00:00<00:00, 49.01it/s]
100%|██████████| 24/24.0 [00:00<00:00, 49.03it/s]
100%|██████████| 24/24.0 [00:00<00:00, 51.24it/s]
100%|██████████| 24/24.0 [00:00<00:00, 51.57it/s]
100%|██████████| 24/24.0 [00:00<00:00, 49.89it/s]
100%|██████████| 24/24.0 [00:00<00:00, 49.36it/s]
100%|██████████| 24/24.0 [00:00<00:00, 49.38it/s]
100%|██████████| 24/24.0 [00:00<00:00, 48.13it/s]
100%|██████████| 24/24.0 [00:00<00:00, 49.73it/s]
100%|██████████| 24/24.0 [00:00<00:00, 50.86it/s]
100%|██████████| 24/24.0 [00:00<00:00, 50.59it/s]
100%|██████████| 24/24.0 [00:00<00:00, 49.16it/s]
100%|██████████| 24/24.0 [00:00<00:00, 49.00it/s]
100%|██████████| 24/24.0 [00:00<00:00, 51.43it/s]
100%|██████████| 24/24.0 [00:00<00:00, 49.62it/s]
100%|██████████| 24/24.0 [00:00<00:00, 50.10it/s]
100%|██████████| 24/24.0 [00:00<00:00, 46.40it/s]
100%|██████████| 24/24.0 [00:00<00:00, 49.09it/s]
100%|██████████| 24/24.0

In [0]:
#!pip install -U albumentations>=0.3.0 --user
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

DATA_DIR_TRAIN = "semantic_drone_dataset/original_images/train/"
ANN_DIR_TRAIN = "semantic_drone_dataset/label_images_semantic/train/"

x_train_dir = os.path.join(DATA_DIR_TRAIN, 'train_crop')
y_train_dir = os.path.join(ANN_DIR_TRAIN, 'train_crop')

DATA_DIR_VAL = "semantic_drone_dataset/original_images/validation/"
ANN_DIR_VAL = "semantic_drone_dataset/label_images_semantic/validation/"

x_valid_dir = os.path.join(DATA_DIR_VAL, 'val_crop')
y_valid_dir = os.path.join(ANN_DIR_VAL, 'val_crop')

DATA_DIR_TEST = "semantic_drone_dataset/original_images/test/"
ANN_DIR_TEST = "semantic_drone_dataset/label_images_semantic/test/"

x_test_dir = os.path.join(DATA_DIR_TEST, 'test_crop')
y_test_dir = os.path.join(ANN_DIR_TEST, 'test_crop')

In [0]:
#!pip install segmentation_models
import segmentation_models as sm
import tensorflow as tf
from tensorflow import keras
from lib.library_seg.library_seg.keras_segmentation.data_utils import data_loader
SM_FRAMEWORK='tf.keras'
sm.set_framework(SM_FRAMEWORK)
tf.keras.backend.set_image_data_format('channels_last')
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
BACKBONE = 'inceptionv3'
preprocess_input = sm.get_preprocessing(BACKBONE)
BATCH_SIZE = 2
CLASSES = ['tree', 'gras', 'other vegetation', 'dirt', 'gravel', 'rocks', 'water', 'paved area', 'pool', 'person', 'dog', 'car', 'bicycle', 'roof', 'wall', 'fence', 'fence-pole', 'window', 'door', 'obstacle']
# define network parameters
n_classes = 1 if len(CLASSES) == 1 else (len(CLASSES) + 1)  # case for binary and multiclass segmentation
activation = 'sigmoid' if n_classes == 1 else 'softmax'
LR = 0.0001
EPOCHS = 5

#create model
model = sm.Unet(BACKBONE, classes=n_classes, activation=activation, input_shape=(1024, 1024, 3))

train_gen = data_loader.image_segmentation_generator(x_train_dir, y_train_dir, BATCH_SIZE, n_classes, input_height=1024, input_width=1024,
                                                            output_height=model.output_shape[1], output_width=model.output_shape[2])
val_gen = data_loader.image_segmentation_generator(x_valid_dir, y_valid_dir, BATCH_SIZE, n_classes, input_height=1024, input_width=1024,
                                                            output_height=model.output_shape[1], output_width=model.output_shape[2])

# define optomizer
optim = keras.optimizers.SGD(learning_rate=LR, momentum=0.9)

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
# set class weights for dice_loss (car: 1.; pedestrian: 2.; background: 0.5;)
#dice_loss = sm.losses.DiceLoss(class_weights=np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 0.5])) 
#focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
#total_loss = dice_loss + (1 * focal_loss)
total_loss = 'categorical_crossentropy'

# actulally total_loss can be imported directly from library, above example just show you how to manipulate with losses
# total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss 

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

# compile keras model with defined optimozer, loss and metrics
model.compile(optim, total_loss, metrics)

# define callbacks for learning rate scheduling and best checkpoints saving
callbacks = [
    keras.callbacks.ReduceLROnPlateau()
]


history = model.fit_generator(train_gen,
                              steps_per_epoch=720,
                              validation_data=val_gen,
                              validation_steps=240,
                              epochs=EPOCHS, callbacks=callbacks,
                              use_multiprocessing=False)

87916544/87910968 [==============================] - 6s 0us/step
Epoch 1/5
720/720 [==============================] - 1522s 2s/step - loss: 2.3431 - iou_score: 0.4350 - f1-score: 0.4384 - val_loss: 1.7075 - val_iou_score: 0.6802 - val_f1-score: 0.6870 - lr: 1.0000e-04
Epoch 2/5
720/720 [==============================] - 1517s 2s/step - loss: 1.9494 - iou_score: 0.6387 - f1-score: 0.6454 - val_loss: 1.4999 - val_iou_score: 0.6709 - val_f1-score: 0.6791 - lr: 1.0000e-04
Epoch 3/5
553/720 [======================>.......] - ETA: 4:46 - loss: 1.8635 - iou_score: 0.6300 - f1-score: 0.6383